In [ ]:
from scrapy import Selector
import requests
import re
import time
import pandas as pd

In [ ]:
product_id_iter = 0
review_id_iter = 0

columns_reviews_df = ['review_id', 'product_id', 'product_name', 'brand', 'user', 'date', 'grade', 'review_title', 
                      'review_body', 'review_pros', 'review_cons']
reviews_df = pd.DataFrame(columns = columns_reviews_df)

top_page_one_url = 'https://wizaz.pl/kosmetyki/pielegnacja-twarzy/kremy-do-twarzy?search_full%5Bsort%5D=reviews_desc&search_full%5Bfilter_reviews_min%5D=&search_full%5Bfilter_hits_min%5D='
top_response = requests.get(top_page_one_url)
top_response = Selector(text = top_response.text)

top_pages_numbers = top_response.xpath('//div[@class = "page-switcher"]/ul/li/a/text()').extract()
if top_pages_numbers:
    top_max_page = int(top_pages_numbers[-1])
else:
    top_max_page =  1

top_page_urls = [top_page_one_url + '&page=' + str(n) for n in range(1, top_max_page+1)]

for top_page_url in top_page_urls:
    
    top_page_url_response = requests.get(top_page_url)
    top_page_url_response = Selector(text = top_page_url_response.text)
    reviews_urls = top_page_url_response.xpath('//h2/a/@href').extract()

    for reviews_url in reviews_urls:
        
        time.sleep(1)

        product_page_response = requests.get(reviews_url)
        product_page_response = Selector( text = product_page_response.text)
        
        ### Extraction of product features
        # number of reviews
        number_of_reviews = int(product_page_response.xpath('//h2/span/text()').extract_first())
        if number_of_reviews == 0:
            continue
        # assigning product id
        product_id = product_id_iter 
        product_id_iter += 1
        # product name
        product_name =  product_page_response.xpath('//h1/text()').extract_first()
        # brand
        brand = product_page_response.xpath(
            '//div[@itemprop = "brand"]/span[@class = "value"]/a/span/text()').extract_first()

        review_max_page = product_page_response.xpath('//div[@class = "page-switcher"]/ul/li/a/text()').extract()
        if review_max_page:
            review_max_page = int(review_max_page[-1])
        else:
            review_max_page = 1
            
        reviews_pages_urls = [reviews_url + '?sortBy=addDate&dir=DESC&page=' + str(n) + '#!#recenzje' \
                                  for n in range(1, review_max_page+1)]
        
        for reviews_page_url in reviews_pages_urls:
            
            reviews_paged_response = requests.get(reviews_page_url)
            reviews_paged_response = Selector(text = reviews_paged_response.text)
            
            ### Extraction of reviews
            # users' names
            users = reviews_paged_response.xpath('//div[@class = "h2"]/a/span/text()').extract()
            # assigning review ids
            review_ids = [x for x in range(review_id_iter, review_id_iter + len(users))]
            review_id_iter = review_ids[-1] + 1
            # date of review
            dates = reviews_paged_response.xpath('//p[@itemprop="datePublished"]/@content').extract()
            # grade
            grades = reviews_paged_response.xpath('//strong[@itemprop="ratingValue"]/text()').extract()
            grades = [int(x) for x in grades]
            # titles of reviews
            review_titles = reviews_paged_response.xpath('//h3/text()').extract()
            # body of review
            review_bodies = ['\n'.join(description.xpath('./text()').extract()) \
                                for description in reviews_paged_response.xpath( \
                                    '//div[@class="opinion"]/p[@itemprop="description"]')]
            # positive aspects of product
            review_pros = [' '.join(description.xpath('./p[@class = "for"]/text()').extract()) \
                               for description in reviews_paged_response.xpath('//div[@class="feedback"]')]
            # negative aspects of product
            review_cons = [' '.join(description.xpath('./p[@class = "against"]/text()').extract()) \
                               for description in reviews_paged_response.xpath('//div[@class="feedback"]')]
            
            product_ids =  [product_id]*len(users)
            product_names = [product_name]*len(users)
            brands = [brand]*len(users)
            
            page_results = pd.DataFrame(zip(review_ids, product_ids, product_names, brands, users, dates, grades, 
                                            review_titles, review_bodies, review_pros, review_cons), 
                                        columns = columns_reviews_df)
            reviews_df = pd.concat([reviews_df, page_results])
        
    print('Page: ' + re.findall('&page=(.*)', top_page_url)[0])

In [ ]:
reviews_df

In [ ]:
reviews_df.shape

In [ ]:
reviews_df.to_excel("data/reviews_full_df.xlsx", index=False, encoding = 'utf-8')